In order to generate a sample of the "cleaned" dataset (no metadata, duplicate observations, outliers etc) I print it on a txt file.

In [31]:
import pandas as pd
import numpy as np

from llama_cpp import Llama

import re

In [2]:
#obtain list of traits

with open('./support_files/trait_id_list.txt', 'r') as openfile:
    trait_ids = openfile.read().splitlines()

print(f"There are {len(trait_ids)} traits in total\n")

#mapping them to int because they were read as strings
trait_ids = list(map(lambda x : int(x), trait_ids))

print(trait_ids)

There are 16 traits in total

[3117, 13, 55, 47, 163, 50, 14, 403, 42, 3106, 33, 1111, 2809, 2807, 2808, 159]


In [3]:
#specifing encoding='latin' is necessary or you get an error
top3_species_top_traits = pd.read_csv('./support_files/top3_species_top_traits.txt', sep='\t', encoding='latin')

print("Total:", len(top3_species_top_traits))

#clear outliers: refer to the documentation obtained with the data request to better understand
#only keeping the measurement that have a value included in a range of 4 standard deviations. The rest are likely outliers or wrong.
#also keep the rows with no value in ErrorRisk, because that is metadata

#500 measurements dropped.
top3_species_top_traits = top3_species_top_traits[(top3_species_top_traits['ErrorRisk'] < 4.0) | (top3_species_top_traits['ErrorRisk'].isnull())]

print("After removing outliers:", len(top3_species_top_traits))

#now remove the duplicates. Again, refer to the documentation for more information
#4032 observations removed
top3_species_top_traits = top3_species_top_traits[top3_species_top_traits['OrigObsDataID'].isnull()]

#Note that duplicates are only referred to trait measurements.
#Therefore removing rows marked as "duplicates" might leave in the dataframe rows related to the single observation (eg. location)
#For these values, TraitName is not present. The information is located in the "DataName" column.
#If exploration is necessary, refer to the "ObservationID" of the original dataframe

print("After removing duplicates:", len(top3_species_top_traits))

#Related to what I mentioned above, now I only keep data related to the trait measurements. No metadata.
trait_measurements = top3_species_top_traits[top3_species_top_traits['TraitID'].isin(trait_ids)]

trait_measurements[:5]

C:\Users\guido\AppData\Local\Temp\ipykernel_18660\1445431305.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  top3_species_top_traits = pd.read_csv('./support_files/top3_species_top_traits.txt', sep='\t', encoding='latin')


Total: 116532
After removing outliers: 116032
After removing duplicates: 112000


,LastName,FirstName,DatasetID,Dataset,SpeciesName,AccSpeciesID,AccSpeciesName,ObservationID,ObsDataID,TraitID,...,Replicates,StdValue,UnitName,RelUncertaintyPercent,OrigObsDataID,ErrorRisk,Reference,Comment,StdValueStr,Unnamed: 28
9,Craine,Joseph,10,Roots Of the World (ROW) Database,Dactylis glomerata,16700,Dactylis glomerata,19150,482854,13.0,...,NaN,458.50000,mg/g,NaN,NaN,0.776812,"Craine, J. M., W. G. Lee, W. J. Bond, R. J. Wi...",C concentration,NaN,NaN
15,Craine,Joseph,10,Roots Of the World (ROW) Database,Dactylis glomerata,16700,Dactylis glomerata,19194,483824,13.0,...,NaN,416.20000,mg/g,NaN,NaN,1.267950,"Craine, J. M., W. G. Lee, W. J. Bond, R. J. Wi...",C concentration,NaN,NaN
20,Craine,Joseph,10,Roots Of the World (ROW) Database,Dactylis glomerata,16700,Dactylis glomerata,19216,484308,14.0,...,NaN,27.80000,mg/g,NaN,NaN,1.177140,"Craine, J. M., W. G. Lee, W. J. Bond, R. J. Wi...",unadjusted N concentration,NaN,NaN
28,Craine,Joseph,130,Global 15N Database,Dactylis glomerata,16700,Dactylis glomerata,19514,491120,42.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"Craine, J. M., A. J. Elmore, M. P. M. Aidar, M...",Growth form,NaN,NaN
29,Craine,Joseph,130,Global 15N Database,Dactylis glomerata,16700,Dactylis glomerata,19514,491121,14.0,...,NaN,15.91335,mg/g,NaN,NaN,2.152440,"Craine, J. M., A. J. Elmore, M. P. M. Aidar, M...",% dry mass,NaN,NaN


In [4]:
# Create an explicit copy of the dataframe with selected columns
trait_measurements_1 = trait_measurements[['AccSpeciesName', 'ObservationID', 'TraitID', 'TraitName', 
                                          'DataName', 'OrigValueStr', 'StdValue', 'UnitName']].copy()

# Create a new column for combined values
# First, convert StdValue to object type to handle mixed types
trait_measurements_1['Value'] = trait_measurements_1['StdValue'].astype(object)

# Replace NaN values with corresponding OrigValueStr values
mask = trait_measurements_1['Value'].isna()
trait_measurements_1.loc[mask, 'Value'] = trait_measurements_1.loc[mask, 'OrigValueStr']

# Now identify non-numerical values in the Value column and set their UnitName to NaN
# This uses pandas' to_numeric with errors='coerce' to identify non-numeric values
non_numeric_mask = pd.to_numeric(trait_measurements_1['Value'], errors='coerce').isna()
trait_measurements_1.loc[non_numeric_mask, 'UnitName'] = np.nan

# Create the final dataframe with desired columns
trait_measurements_unified = trait_measurements_1[['AccSpeciesName', 'ObservationID', 'TraitID', 
                                                 'TraitName', 'DataName', 'Value', 'UnitName']].copy()

# Rename Value to StdValue if you want to keep the original column name
trait_measurements_unified = trait_measurements_unified.rename(columns={'Value': 'StdValue'})

trait_measurements_unified['TraitID'] = trait_measurements_unified['TraitID'].map(lambda x : int(x))

trait_measurements_unified

,AccSpeciesName,ObservationID,TraitID,TraitName,DataName,StdValue,UnitName
9,Dactylis glomerata,19150,13,Leaf carbon (C) content per leaf dry mass,Leaf carbon content per dry mass,458.5,mg/g
15,Dactylis glomerata,19194,13,Leaf carbon (C) content per leaf dry mass,Leaf carbon content per dry mass,416.2,mg/g
20,Dactylis glomerata,19216,14,Leaf nitrogen (N) content per leaf dry mass,Leaf nitrogen content per dry mass (Nmass),27.8,mg/g
28,Dactylis glomerata,19514,42,Plant growth form,Plant growth form,Herbaceous Monocot,NaN
29,Dactylis glomerata,19514,14,Leaf nitrogen (N) content per leaf dry mass,Leaf nitrogen content per dry mass (Nmass),15.91335,mg/g
...,...,...,...,...,...,...,...
116527,Dactylis glomerata,6487798,50,Leaf nitrogen (N) content per leaf area,Leaf nitrogen content per area (Narea),1.152584,g m-2
116528,Dactylis glomerata,6487799,50,Leaf nitrogen (N) content per leaf area,Leaf nitrogen content per area (Narea),0.718069,g m-2
116529,Dactylis glomerata,6487800,50,Leaf nitrogen (N) content per leaf area,Leaf nitrogen content per area (Narea),0.836561,g m-2
116530,Dactylis glomerata,6487801,50,Leaf nitrogen (N) content per leaf area,Leaf nitrogen content per area (Narea),0.971015,g m-2


In [5]:
trait_measurements_unified.to_csv('./support_files/cleaned_top3_species_top_traits.txt', sep='\t', index=False)

#Then I will manually pick a subset of significant entries...

In [ ]:
#this was used to test the prompt of the LLM and check if everything worked correctly.

sample_measurements = pd.read_csv('.\\support_files\\SAMPLE_top3_species_top_traits.txt', sep='\t', encoding='latin')

# sample_measurements.info()

#pd.duplicated creates a mask of duplicated rows. Pick only the subset of Obs and TraitID, and mark all duplicates as True (not only the first)
# duplicates = sample_measurements[sample_measurements.duplicated(subset=['ObservationID', 'TraitID'], keep=False)]
# duplicates[:5]
# grouped = duplicates.groupby(["ObservationID", "TraitID"])

#do not include only the duplicates in this case, let's test also what happens with single rows
# grouped = sample_measurements.groupby(["ObservationID", "TraitID"])

# group_keys = list(grouped.groups.keys())

# # Sample a fraction of group keys, e.g., 20%
# sampled_keys = pd.Series(group_keys).sample(frac=0.2, random_state=1)

# sampled_keys_set = set(sampled_keys)


# # Filter original dataframe to keep only rows belonging to the sampled groups
# sampled_df = trait_measurements_unified[
#     trait_measurements_unified.set_index(["ObservationID", "TraitID"]).index.isin(sampled_keys_set)
# ].reset_index()

# sampled_df.head()


# print(len(grouped))

# #Show a sample group
# for (obs_id, trait), group in grouped:
#     print(f"ObservationID: {obs_id}, Trait: {trait}")
#     print(group[["DataName", "StdValue"]])
#     break  # just show one example for now

[(25163, 14), (25163, 42), (28738, 14), (28739, 14), (28740, 14), (807324, 55), (807334, 3117), (807523, 33), (807524, 33), (807525, 33), (941113, 13), (941113, 14), (941113, 42), (941113, 50), (941113, 403), (941113, 3106), (941959, 42), (941959, 3106), (1043810, 42), (1043819, 42), (1259027, 3117)]
{(1043810, 42), (28739, 14), (941113, 3106), (941113, 13)}


In [63]:
#Now I will test it with an actual sample of the original dataframe

#prints the number of groups for each size
print(trait_measurements_unified.groupby(["ObservationID", "TraitID"]).size().value_counts())


grouped = trait_measurements_unified.groupby(["ObservationID", "TraitID"])

group_keys = list(grouped.groups.keys())

# Sample a fraction of group keys, e.g., 20%
sampled_keys = pd.Series(group_keys).sample(frac=0.05, random_state=42)

# Convert to a set for faster filtering
sampled_keys_set = set(sampled_keys)

# Filter original dataframe to keep only rows belonging to the sampled groups
sampled_df = trait_measurements_unified[
    trait_measurements_unified.set_index(["ObservationID", "TraitID"]).index.isin(sampled_keys_set)
].reset_index()

print(sampled_df.groupby(["ObservationID", "TraitID"]).size().value_counts())

#grouping by after the previous dataframe was ungrouped for sampling
sample_groups = sampled_df.groupby(['ObservationID', 'TraitID'])

print(len(sample_groups))


1     13161
2      1469
3        96
4         4
13        2
Name: count, dtype: int64
1    656
2     77
3      3
4      1
Name: count, dtype: int64
737


In [7]:
MODEL_PATH = "./models/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

llm = Llama(model_path=MODEL_PATH, verbose=False, chat_format='llama-2')

# Simple test prompt
response = llm("Q: What is the capital of France?\nA:", max_tokens=32)

# print(response)

print(response["choices"][0]["text"])


llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


 Paris


In [ ]:
CATEGORICAL_TRAITS = {
    "Plant growth form",
    "Seedbank type",
    "Seed (seedbank) longevity",
}

#Only for categorical traits, include the value of the trait, to help the model choose better the row to keep

def build_user_content(trait_name, data_names, categoricals=None):
    """Return a prompt to select the row that best matches the trait name 
    (given the data description/context + value of categorical traits),
      when there are multiple measurement of the same trait
    regarding a single observation.
    """

    if categoricals is not None:
        # include values
        numbered = "\n".join(
            [f"{i+1}. {name} = {val}" for i, (name, val) in enumerate(zip(data_names, categoricals))]
        )
    else:
        numbered = "\n".join([f"{i+1}. {name}" for i, name in enumerate(data_names)])
    
    return (
        f"Given the following versions of the same trait:\n{numbered}\n\n"
        f"Which one best represents the trait name: \"{trait_name}\"?\n"
        f"Respond only with the number of the best matching trait version (e.g., '1'). Do not write anything else."
    )

In [64]:
selected_rows = []

#iterate over the rows with the same ObservationID
#group is a dataframe of all rows sharing ObsID
for obs_id, group in sample_groups:
    data_names = group['DataName'].to_list()
    trait_name = group['TraitName'].iloc[0] #they are all the same, the first is fine

    #if the observation has a single data name, keep it
    if len(data_names)==1:
        selected = data_names[0]
    else: 

         # Check if the trait is categorical
        is_categorical = trait_name in CATEGORICAL_TRAITS

        categoricals = group['StdValue'].to_list() if is_categorical else None

        messages = [
            {"role": "system", "content": "You are a helpful assistant. When given multiple versions of a trait, "
                "you must select the version that best matches the given trait name. "
                "Respond ONLY with the number corresponding to your choice. Do not explain your answer."},
            {"role": "user", "content": None}
        ]
        content = build_user_content(trait_name, data_names, categoricals)
        messages[1]['content'] = content
        response = llm.create_chat_completion(messages)
        answer = response["choices"][0]["message"]["content"].strip()

        print("\nPROMPT: ", content)
        print("\nANSWER: ", answer)

        match = re.search(r"\b(\d+)\b", answer)
        # \b(\d+)\b is a regex pattern:
        # \d+ matches one or more digits (the number),
        # \b ensures that the digits are bounded (i.e., not part of a word like "version1").
        # If it finds a match, match.group(1) extracts the number string (e.g., '2').
        if match:
            index = int(match.group(1)) - 1
            if 0 <= index < len(data_names):
                selected = data_names[index]
        else:
            selected = None

    # Add the full row with all original columns
    if selected is not None:
        match = group[group['DataName'] == selected]
        if not match.empty:
            selected_rows.append(match.iloc[0])  # take the full row
            

final_df = pd.DataFrame(selected_rows)


PROMPT:  Given the following versions of the same trait:
1. Whole plant aboveground vegetative dry mass per individual plant
2. Shoot dry mass per plant

Which one best represents the trait name: "Plant biomass and allometry: Shoot dry mass (plant aboveground dry mass) per plant"?
Respond only with the number of the best matching trait version (e.g., '1'). Do not write anything else.

ANSWER:  1.

PROMPT:  Given the following versions of the same trait:
1. Plant growth form = Graminoid
2. growth Form = Bunch
3. Low Growing Grass = No
4. Shape and Orientation = Erect

Which one best represents the trait name: "Plant growth form"?
Respond only with the number of the best matching trait version (e.g., '1'). Do not write anything else.

ANSWER:  1.

PROMPT:  Given the following versions of the same trait:
1. SLA: undefined if petiole in- or excluded
2. SLA: undefined if petiole in- or excluded (1)

Which one best represents the trait name: "Leaf area per leaf dry mass (specific leaf area,

In [65]:
final_df

,index,AccSpeciesName,ObservationID,TraitID,TraitName,DataName,StdValue,UnitName
0,73,Dactylis glomerata,23672,42,Plant growth form,Plant growth form,Herbaceous Monocot,NaN
1,175,Achillea millefolium,28740,14,Leaf nitrogen (N) content per leaf dry mass,Leaf nitrogen content per dry mass (Nmass),13.897986,mg/g
2,231,Achillea millefolium,29085,14,Leaf nitrogen (N) content per leaf dry mass,Leaf nitrogen content per dry mass (Nmass),20.5,mg/g
3,334,Trifolium pratense,93264,42,Plant growth form,Plant growth form,herb,NaN
4,545,Achillea millefolium,94198,42,Plant growth form,Plant growth form,herbaceous dicotyl,NaN
...,...,...,...,...,...,...,...,...
798,114469,Dactylis glomerata,6487781,3106,Plant height vegetative,Plant height vegetative,0.725333,m
799,114491,Dactylis glomerata,6487783,3106,Plant height vegetative,Plant height vegetative,0.500625,m
800,114669,Dactylis glomerata,6487799,14,Leaf nitrogen (N) content per leaf dry mass,Leaf nitrogen content per dry mass (Nmass),15.788701,mg/g
821,116529,Dactylis glomerata,6487800,50,Leaf nitrogen (N) content per leaf area,Leaf nitrogen content per area (Narea),0.836561,g m-2


In [66]:
#POSSIBLY To do next: compare with something else (change temperature or idk) and check if there is any difference in the two datasets.
# Also, clustering of the plant growth forms to see how many are there. 